# Covid Geographic Visualizations

In [1]:
import pandas as pd
import pickle
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
from datetime import datetime

#### Import Data

In [2]:
with open('covid_data.p', 'rb') as f:
    data = pickle.load(f)

#### Feature Engineering

In [3]:
def engineerFeatures(df):
    df = df[df.deaths_cdc > 0]
    df = df.dropna(subset=['lat_ea','long_ea'])
    df.lat_ea = pd.to_numeric(df.lat_ea)
    df.long_ea = pd.to_numeric(df.long_ea)
    
    df["deaths_per_100k_ave7"]=df.deaths_per_100k.rolling(window=7).mean()
    df["deaths_per_100k_ave14"]=df.deaths_per_100k.rolling(window=14).mean()
    df["deaths_per_100k_ave21"]=df.deaths_per_100k.rolling(window=21).mean()
    
    df["daily_deaths_per_100k"]=df.deaths_per_100k.diff()
    df["daily_deaths_per_100k_ave7"]=df.deaths_per_100k.diff().rolling(window=7).mean()
    df["daily_deaths_per_100k_ave14"]=df.deaths_per_100k.diff().rolling(window=14).mean()
    df["daily_deaths_per_100k_ave30"]=df.deaths_per_100k.diff().rolling(window=30).mean()
    
    col = df[["retail_and_recreation_percent_change_from_baseline_goog_cnty",
                    "workplaces_percent_change_from_baseline_goog_cnty"]].apply(pd.to_numeric)
    df['mobility_change'] = col.mean(axis=1)
    
    df['mobility_change_ave7'] = df['mobility_change'].rolling(window=7).mean()
    df['mobility_change_ave14'] = df['mobility_change'].rolling(window=14).mean()
    df['mobility_change_ave30'] = df['mobility_change'].rolling(window=30).mean()+100
    
    df=df.dropna(subset=[
        #'deaths_per_100k',
        #'deaths_per_100k_ave7',
        'deaths_per_100k_ave14',
        #'deaths_per_100k_ave21',
        #'daily_deaths_per_100k',
        #'daily_deaths_per_100k_ave7',
        #'daily_deaths_per_100k_ave14',
        'daily_deaths_per_100k_ave30',
        #'mobility_change','mobility_change_ave7','mobility_change_ave14','mobility_change_ave30'
    ])
    
    return df

In [4]:
data = engineerFeatures(data)

In [5]:
def plotMap(df, metric, dt, cmap='Reds', min_death_count = 50, scale_factor=100, outfile="img/test.jpg",
           area = [-125, -65, 25, 47], dpi=40):
    
    df = df[df.date == dt]
    df = df[df.deaths_cdc >= min_death_count]
    df = df[df[metric] > 0]
    
    fig = plt.figure(figsize=(20, 10))
   
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_title("Date: " + str(dt) + "  -  " + datetime.strptime(dt, "%Y-%m-%d").strftime("%B"))

    ax.add_feature(cfeature.LAND.with_scale('50m'), color="black", alpha=0.8)
    ax.add_feature(cfeature.OCEAN.with_scale('50m'), color="black", alpha=0.9)
    ax.add_feature(cfeature.LAKES.with_scale('50m'), color="black", alpha=0.1)
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    plt.scatter(x=df.long_ea, y=df.lat_ea, 
                c=df[metric]*scale_factor,
                cmap=cmap,
                s=df[metric]*scale_factor,
                alpha=0.5)

    ax.coastlines('50m')

    ax.set_extent(area)
    
    fig.savefig(outfile, bbox_inches='tight', dpi=dpi)
    plt.close(fig)

In [6]:
def createImages(folder, metric, scale_factor, testmode=False, area = [-125, -65, 25, 47], dpi=40, min_death_count=15):
    i=0
    dates = pd.to_datetime(data['date'].sort_values()).dt.date.unique()
    for d in dates:
        outfile = folder +'/'+ str(i) + '_img.jpg'
        plotMap(data, 
                metric=metric, 
                dt=d.strftime("%Y-%m-%d"), 
                cmap='Reds', 
                min_death_count=min_death_count, 
                scale_factor=scale_factor, 
                outfile=outfile,
                area=area,
                dpi=dpi)
        i+=1
        if testmode:
            break
    return

In [7]:
createImages('img1','deaths_cdc', 3)

In [8]:
createImages('img2','deaths_per_100k_ave14', 7)

In [9]:
createImages('img3','mobility_change_ave30', 3, min_death_count=0)

In [10]:
createImages('img4','daily_deaths_per_100k_ave30', 350)

In [11]:
# Northeast
createImages('img5','deaths_per_100k_ave14', 10, area = [-81, -67, 38, 48], dpi = 70)

In [12]:
# Northeast
createImages('img6','daily_deaths_per_100k_ave30', 150, area = [-81, -67, 38, 48], dpi = 70)

In [13]:
# Southeast
createImages('img7','deaths_per_100k_ave14', 10, area = [-95, -75, 25, 38], dpi = 70)

In [14]:
# Southeast
createImages('img8','daily_deaths_per_100k_ave30', 150, area = [-95, -75, 25, 38], dpi = 70)

In [15]:
# Mid West
createImages('img9','deaths_per_100k_ave14', 10, area = [-105, -80, 36, 48], dpi = 70)

In [16]:
# Mid West
createImages('img10','daily_deaths_per_100k_ave30', 150, area = [-105, -80, 36, 48], dpi = 70)

In [17]:
# West
createImages('img11','deaths_per_100k_ave14', 10, area = [-125, -104, 30, 50], dpi = 70)

In [18]:
# West
createImages('img12','daily_deaths_per_100k_ave30', 150, area = [-125, -104, 30, 50], dpi = 70)

In [19]:
# Texas
createImages('img13','deaths_per_100k_ave14', 10, area = [-107, -89, 25.7, 41], dpi = 70)

In [20]:
# Texas
createImages('img14','daily_deaths_per_100k_ave30', 150, area = [-107, -89, 25.7, 41], dpi = 70)